In [ ]:
from tensorflow.keras import backend as K
from tensorflow import keras

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib import pyplot

import threading
import argparse
import pickle
import numpy as np
import pandas as pd
import csv
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as metrics
import math
import time
import gc
from datetime import datetime
from collections import Iterable
from tensorflow.keras import losses
from statistics import mean
from tensorflow.keras.utils import plot_model
import tensorflow.python.keras.backend as K

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint, EarlyStopping,Callback
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.optimizers import *
#K.set_image_data_format('channels_first')

import seaborn as sns
sns.set()

import os


import tensorflow.compat.v1 as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))


start_time = time.time()

n_channels = 1
n_classes = 18
sample_batch = 10
time_step = 1
time_step2 = 3
number_stations = 18
#optim = Adam(lr = 0.0000, decay = 0.000015)
optim = Adam(learning_rate = 1e-4, decay = 1e-4)
# Setup the model


def eval_metrics_on(predictions,labels):
    '''assuming this is a regression task; labels are continuous-valued floats
    
    returns most regression-related scores for the given predictions/targets as a dictionary:
    
        r2, mean_abs_error, mse, rmse, median_absolute_error, explained_variance_score
    '''
    #if len(labels[0])==2: #labels is list of data/labels pairs
        #labels = np.concatenate([l[1] for l in labels])
    #predictions = predictions[:,0]
    prdictions = np.array(predictions)
    labels = np.array(labels)
    
    r2                       = metrics.r2_score(labels, predictions)
    mean_abs_error           = np.abs(predictions - labels).mean()
    mse                      = ((predictions - labels)**2).mean()
    rmse                     = np.sqrt(mse)
    #median_absolute_error    = metrics.median_absolute_error(labels, predictions) # robust to outliers
    explained_variance_score = metrics.explained_variance_score(labels, predictions) # best score = 1, lower is worse
    return {'r2':r2, 'mean_abs_error':mean_abs_error, 'mse':mse, 'rmse':rmse, 
    'explained_variance_score':explained_variance_score}


def predict_graph(true_val,predict_val):
    for sn in range(number_stations):
        plt.figure(figsize=(6,3))
        plt.figure(sn+1)
        plt.title('_next_1day station = ' + str(sn+1))
        plt.plot(true_val[:,sn::number_stations], c='g', label= 'gt')
        plt.plot(predict_val[:,sn::number_stations], c='r', label= 'pred')
        plt.legend(loc='best',prop={'size': 5})
        plt.legend(loc='best')
        name = str("firstgroup_stat_") + str(sn) + str(".png")
        plt.show()

#This function used to clear the memory         
class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        K.clear_session()

def gui_generator(tile28v05,tile28v06,tile29v05,tile29v06,remoteweather,pm,y_lst,batch_size,num_channels,
                  num_channels_2,num_channels_3,
                  time,time_label,cover_hr_in=1):
    while True:
        x_shape_1 = (batch_size,time_label,300,300,num_channels)
        
        x_shape_2 = (batch_size,time_label,300,300,num_channels)
        
        x_shape_3 = (batch_size,time,4,7,num_channels_2)
        
        y_shape = (batch_size,time,18)
        
        h28v05 = np.zeros(shape=x_shape_1)
        h28v06 = np.zeros(shape=x_shape_1)
        
        h29v05 = np.zeros(shape=x_shape_2)
        h29v06 = np.zeros(shape=x_shape_2)
        
        h31v06 = np.zeros(shape=x_shape_3)
        #h32v06 = np.zeros(shape=x_shape_4)
        pm_batch = np.zeros(shape=y_shape)
        
        
        #pm = np.zeros(shape=y_shape)
        y_batch = np.zeros(shape=y_shape)
        
        end = y_lst.shape[0]
        #print('this is end ', end)
        
        
        for batch_idx in range(0,end,int(batch_size)):
            for seq_idx in range(int(batch_size)):
                idx = batch_idx + seq_idx
                
                h28v05[seq_idx] = tile28v05[idx:idx+cover_hr_in][0]
                h28v06[seq_idx] = tile28v06[idx:idx+cover_hr_in][0]
                
                h29v05[seq_idx] = tile29v05[idx:idx+cover_hr_in][0]
                h29v06[seq_idx] = tile29v06[idx:idx+cover_hr_in][0]
                
                h31v06[seq_idx] = remoteweather[idx:idx+cover_hr_in][0]
                pm_batch[seq_idx] = pm[idx:idx+cover_hr_in][0]
                
                y_batch[seq_idx] = y_lst[idx:idx+cover_hr_in][0]

            yield ([h28v05,h28v06,h29v05,h29v06,h31v06,pm_batch],y_batch)
            
            
def pred_generat(tile28v05,tile28v06,tile29v05,tile29v06,remoteweather,pm,batch_size,num_channels,
                  num_channels_2,num_channels_3,
                  time,time_label,cover_hr_in=1):
    while True:
        x_shape_1 = (batch_size,time_label,300,300,num_channels)
        
        x_shape_2 = (batch_size,time_label,300,300,num_channels)
        
        x_shape_3 = (batch_size,time,4,7,num_channels_2)
        
        y_shape = (batch_size,time,18)
        
        
        h28v05 = np.zeros(shape=x_shape_1)
        h28v06 = np.zeros(shape=x_shape_1)
        
        h29v05 = np.zeros(shape=x_shape_2)
        h29v06 = np.zeros(shape=x_shape_2)
        
        h31v06 = np.zeros(shape=x_shape_3)
        pm_batch = np.zeros(shape=y_shape)
        
        
        #pm = np.zeros(shape=y_shape)
        y_batch = np.zeros(shape=y_shape)
        
        end = pm.shape[0]
        #print('this is end ', end)
        
        
        for batch_idx in range(0,end,int(batch_size)):
            for seq_idx in range(int(batch_size)):
                idx = batch_idx + seq_idx
                
                h28v05[seq_idx] = tile28v05[idx:idx+cover_hr_in][0]
                h28v06[seq_idx] = tile28v06[idx:idx+cover_hr_in][0]
                
                h29v05[seq_idx] = tile29v05[idx:idx+cover_hr_in][0]
                h29v06[seq_idx] = tile29v06[idx:idx+cover_hr_in][0]
                
                h31v06[seq_idx] = remoteweather[idx:idx+cover_hr_in][0]
                pm_batch[seq_idx] = pm[idx:idx+cover_hr_in][0]
                #batch_h32v06[seq_idx] = tile32v06[idx:idx+cover_hr_in][0]

            
            yield ([h28v05,h28v06,h29v05,h29v06,h31v06,pm_batch])
            


def fn_model_schw3(input_data):
    #model_one_conv_1 = Permute((2,1,3,4))(input_data)
    model_one_conv_1 = AveragePooling3D(pool_size=(1,2,2), strides=(1,2,2),padding='same')(input_data)
    #model_one_conv_1 = AveragePooling3D(pool_size=(1,2,2), strides=(1,2,2),padding='same', data_format='channels_first')(model_one_conv_1)
    model_one_conv_1 = TimeDistributed(Conv2D(32, (3,3),strides =(2,2), activation='relu'))(model_one_conv_1)
    #model_one_conv_1 = TimeDistributed(AveragePooling2D(pool_size=(2,4), strides=(1,1),padding='same',data_format='channels_first'))(model_one_conv_1)
    model_one_conv_1 = TimeDistributed(Conv2D(32, (3,3) ,activation='relu',padding = 'same'))(model_one_conv_1)
    

    model_one_conv_1 = ConvLSTM2D(32, (3, 3),padding='same', activation='relu',kernel_regularizer=L1L2(l1 = 0.01,l2=0.01),return_sequences=True)(model_one_conv_1)
    #model_one_conv_1 = AveragePooling3D(pool_size=(1,3,3), strides=(1,2,2),padding='same',data_format='channels_first')(model_one_conv_1)
    model_one_conv_1 = BatchNormalization()(model_one_conv_1)
    #model_one_conv_1 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2),padding='same')(model_one_conv_1)
    model_one_conv_1 = ConvLSTM2D(32, (3, 3), padding='same', activation='relu',kernel_regularizer=L1L2(l1 = 0.01,l2=0.01),return_sequences=True)(model_one_conv_1)
    
    model_one_conv_1 = AveragePooling3D(pool_size=(1,10,10), strides=(1,10,10),padding='same')(model_one_conv_1)
    
    
    return model_one_conv_1

    
    
def weather_model_schw(input_data):
    model_one_conv_1 = ConvLSTM2D(64, (1, 3),padding='same', activation='relu',kernel_regularizer=L1L2(l1 = 0.01,l2=0.01),return_sequences=True)(input_data)
    #model_one_conv_1 = TimeDistributed(Conv2D(32, (1,3), activation='relu',data_format='channels_first'))(input_data)
    model_one_conv_1 = BatchNormalization()(model_one_conv_1)
    #model_one_conv_1 = TimeDistributed(Conv2D(64, (1,3) ,activation='relu',padding = 'same',data_format='channels_first'))(model_one_conv_1)
    #model_one_conv_1 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2),padding='same')(model_one_conv_1)
    model_one_conv_1 = ConvLSTM2D(42, (1, 3), padding='same', activation='relu',kernel_regularizer=L1L2(l1 = 0.01,l2=0.01), return_sequences=True)(model_one_conv_1)
    #model_one_conv_1 = Flatten()(input_data)
    #model_one_conv_1 = RepeatVector(time_step)(model_one_conv_1)
    #model_one_conv_1 = LSTM(64, activation='relu',dropout = 0.3,recurrent_dropout=0.3,return_sequences=True)(model_one_conv_1)
    
    return model_one_conv_1
    


# tile 1
data_shape1 = (1,300,300,1)
in1 = Input(shape=data_shape1)
model1 = fn_model_schw3(in1)

# tile 2
data_shape2 = (1,300,300,1)
in2 = Input(shape=data_shape2)
model2 = fn_model_schw3(in2)

# tile 3
data_shape3 = (1,300,300,1)
in3 = Input(shape=data_shape3)
model3 = fn_model_schw3(in3)

# tile 4
data_shape4 = (1,300,300,1)
in4 = Input(shape=data_shape4)
model4 = fn_model_schw3(in4)

#weather model
weath_data_shape = (3,4,7,148)
in5 = Input(shape=weath_data_shape, name='weath_input')
model5 = weather_model_schw(in5)

#input for PM2.5 as feature
data_shape = (3,18)
in6 = Input(shape=data_shape)
#model6 = pm25_model(in6)

tile1_yr14 = np.transpose(np.load('h28v05_downscale_yr14_n3day_traindata.npy')[9:].astype('float'),axes = (0,1,3,4,2))
tile2_yr14 = np.transpose(np.load('h28v06_downscale_yr14_n3day_traindata.npy')[9:].astype('float'),axes = (0,1,3,4,2))
tile3_yr14 = np.transpose(np.load('h29v05_downscale_yr14_n3day_traindata.npy')[9:].astype('float'),axes = (0,1,3,4,2))
tile4_yr14 = np.transpose(np.load('h29v06_downscale_yr14_n3day_traindata.npy')[9:].astype('float'),axes = (0,1,3,4,2))

tile1_yr15 = np.transpose(np.load('h28v05_downscale_yr15_n3day_traindata.npy')[9:].astype('float'),axes = (0,1,3,4,2))
tile2_yr15 = np.transpose(np.load('h28v06_downscale_yr15_n3day_traindata.npy')[9:].astype('float'),axes = (0,1,3,4,2))
tile3_yr15 = np.transpose(np.load('h29v05_downscale_yr15_n3day_traindata.npy')[9:].astype('float'),axes = (0,1,3,4,2))
tile4_yr15 = np.transpose(np.load('h29v06_downscale_yr15_n3day_traindata.npy')[9:].astype('float'),axes = (0,1,3,4,2))

tile1_train = np.concatenate((tile1_yr14,tile1_yr15),axis = 0)
tile2_train = np.concatenate((tile2_yr14,tile2_yr15),axis = 0)
tile3_train = np.concatenate((tile3_yr14,tile3_yr15),axis = 0)
tile4_train = np.concatenate((tile4_yr14,tile4_yr15),axis = 0)

weather_yr14 = np.transpose(np.load('remote_weather_yr14_n3day_train.npy')[9:].astype('float'),axes = (0,1,3,4,2))
weather_yr15 = np.transpose(np.load('remote_weather_yr15_n3day_train.npy')[9:].astype('float'),axes = (0,1,3,4,2))

remote_weather_train = np.concatenate((weather_yr14,weather_yr15),axis = 0)

local_pm = np.load('pm_n3day_daily_train.npy')[:-6].astype('float')
tru_val = np.load('pm_label_n3day_daily_train.npy')[:-6].astype('float')


tile1_test = np.transpose(np.load('h28v05_downscale_yr16_n3day_testdata.npy')[9:].astype('float'),axes = (0,1,3,4,2))
tile2_test = np.transpose(np.load('h28v06_downscale_yr16_n3day_testdata.npy')[9:].astype('float'),axes = (0,1,3,4,2))
tile3_test = np.transpose(np.load('h29v05_downscale_yr16_n3day_testdata.npy')[9:].astype('float'),axes = (0,1,3,4,2))
tile4_test = np.transpose(np.load('h29v06_downscale_yr16_n3day_testdata.npy')[9:].astype('float'),axes = (0,1,3,4,2))


local_pm_test = np.load('pm_n3day_daily_valid.npy')[:-4].astype('float')
tru_val_test = np.load('pm_label_n3day_valid_daily.npy')[:-4].astype('float')
remote_weather_test = np.transpose(np.load('remote_weather_n3day_test.npy')[9:].astype('float'),axes = (0,1,3,4,2))

train_sample = remote_weather_train[:-6].shape[0]
valid_sample = remote_weather_test[:-4].shape[0]

chan = 1
chan2 = 148
chan3 = 4
time_t = 3
sample_per_epoch = train_sample/sample_batch
sample_per_valid = valid_sample/sample_batch


print('sample per epoch in train',sample_per_epoch, 'sample per epoch in valid',sample_per_valid)

model_final1 = Flatten()(model1)
model_final2 = Flatten()(model2)
model_final3 = Flatten()(model3)
model_final4 = Flatten()(model4)


model_final_concat_allAOD = concatenate([model_final1,model_final2,model_final3,model_final4])
#model_final_concat_allAOD = concatenate([model_final3,model_final4])


#weather_model = TimeDistributed(Flatten())(model5)
weather_model = Flatten()(model5)
weather_model = RepeatVector(time_step2)(weather_model)

model_final_concat = RepeatVector(time_step2)(model_final_concat_allAOD)
model_final_concat = concatenate([model_final_concat,weather_model])

model_pm = Flatten()(in6)
model_pm = RepeatVector(time_step2)(model_pm)

#model_final_concat = concatenate([model_final_concat,model6])
model_final_concat = concatenate([model_final_concat,model_pm])


model_final_concat = TimeDistributed(Dense(64, activation='relu',bias_regularizer=L1L2(l1 = 0.01,l2=0.01)))(model_final_concat)
#model_final_concat = Dense(32, activation='relu',bias_regularizer=L1L2(l1 = 0.01,l2=0.01))(model_final_concat)
model_final_concat = TimeDistributed(Dropout(rate = 0.4))(model_final_concat)
#model_final_concat = Dropout(rate = 0.4)(model_final_concat)
model_final_concat = TimeDistributed(Dense(42, activation='relu',bias_regularizer=L1L2(l1 = 0.01,l2=0.01)))(model_final_concat)
#model_final_concat = Dense(32, activation='relu',bias_regularizer=L1L2(l1 = 0.01,l2=0.01))(model_final_concat)
model_final_concat = TimeDistributed(Dropout(rate=0.4))(model_final_concat)

model_final_concat = TimeDistributed(Dense(n_classes))(model_final_concat)

model = Model(inputs=[in1,in2,in3,in4,in5,in6],outputs=model_final_concat)

model.summary()

NB_EPOCH = 3

model.compile(optimizer = optim, loss = 'mse', metrics = ['mse'], run_eagerly=True)

keras_callback = [
                EarlyStopping(monitor='val_mse', patience = 180, mode='min'),
                ModelCheckpoint(filepath='next3day_4tile_18stations.h5', monitor='val_mse', 
                save_best_only=True,mode='min',verbose=1)
                ]

        
history = model.fit(gui_generator(tile1_train[:-6],tile2_train[:-6],tile3_train[:-6],tile4_train[:-6],
                                  remote_weather_train[:-6],local_pm,tru_val,sample_batch,chan,
                                            chan2,chan3,time_t,time_step),
                                              steps_per_epoch = sample_per_epoch,
                                            epochs = NB_EPOCH,workers=0,shuffle = False,
                              validation_data = gui_generator(tile1_test[:-4],tile2_test[:-4],tile3_test[:-4],tile4_test[:-4],
                                            remote_weather_test[:-4],local_pm_test,
                                                              tru_val_test,
                                            sample_batch,chan,chan2,chan3,time_t,time_step),
                                            validation_steps = sample_per_valid,verbose = 1,callbacks=[keras_callback, ClearMemory()])



#summarize histoy for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'],loc = 'upper left')
plt.show()

print('this is val_mse')
plt.plot(history.history['mse'])
plt.plot(history.history['val_mse'])
plt.title('model mse')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.legend(['train','test'],loc = 'upper left')
plt.show()


predictions_edit = model.predict(pred_generat(tile1_test[:-4],tile2_test[:-4],tile3_test[:-4],tile4_test[:-4],
                                            remote_weather_test[:-4],local_pm_test,
                            sample_batch,chan,chan2,chan3,time_t,time_step),
                                 steps = sample_per_valid, verbose=1,workers=0)
        

predictions = predictions_edit.reshape(predictions_edit.shape[0]*predictions_edit.shape[1],predictions_edit.shape[2])
print(predictions_edit.shape)
y_label = np.load('pm_label_n3day_valid_daily.npy')[:-4].astype("float")
#y_label = np.load('label_valid.npy')
y_label2 = y_label.reshape(y_label.shape[0]*y_label.shape[1],y_label.shape[2])
print('shape of true and predict ',y_label2.shape,predictions.shape)

#perform model evaluation
eval_results = eval_metrics_on(predictions,y_label2)
print('this is evaluation results for r2,mean_abs_error,mse,rmse,expl_var_score ',eval_results)
#print('this is evaluation results for r2,mean_abs_error,mse,rmse,expl_var_score ',eval_results)

#observe the graph between prediction and true value of PM2.5 for last hour
y_true = y_label[:,2:3,:]
y_true = y_true.reshape(y_true.shape[0]*y_true.shape[1],y_true.shape[2])
pred_graph = predictions_edit[:,1:2,:]
#pred_graph = predictions_edit
#pred_graph = predictions_edit.reshape(predictions_edit.shape[0]*predictions_edit.shape[1],predictions_edit.shape[2])
pred_graph =pred_graph.reshape(pred_graph.shape[0]*pred_graph.shape[1],pred_graph.shape[2])
predict_graph(y_true,pred_graph)

print('rmse value')
stat_dict = {}
stat_avg = []
for i in range(time_step2):
    t1 = predictions_edit[:,i:i+1,:]
    t1 = t1.reshape(t1.shape[0]*t1.shape[1],t1.shape[2])
    t2 = y_label[:,i:i+1,:]
    t2 = t2.reshape(t2.shape[0]*t2.shape[1],t2.shape[2])
    rsl = []
    for sn in range(number_stations):
        name = 'stats_' + str(sn)
        score2 = np.sqrt(np.mean(np.square(t1[:,sn::number_stations]-t2[:,sn::number_stations])))
        stat_dict.setdefault(name,[]).append(score2)
        #print(score2)
        rsl.append(score2)
for k in  stat_dict.keys():
    avg_value = sum(stat_dict[k])/time_step2
    #print(avg_value)
    stat_avg.append(avg_value)
print('mean of all stations ',mean(stat_avg))

In [ ]:
layer_name = 'concatenate_1'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)

predictions_edit = intermediate_layer_model.predict(pred_generat(tile1_test[:-4],tile2_test[:-4],tile3_test[:-4],tile4_test[:-4],
                                            remote_weather_test[:-4],local_pm_test,
                            sample_batch,chan,chan2,chan3,time_t,time_step),
                                 steps = sample_per_valid, verbose=1,workers=0)



print(predictions_edit.shape)
np.save('middlelayer_test_3day_4tile.npy',predictions_edit)
#np.save('prediction14_4hr_ver2.npy',pred_graph)

In [ ]:
layer_name = 'concatenate_1'
intermediate_layer_model2 = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)

predictions_edit = intermediate_layer_model2.predict(pred_generat(tile1_train[:-6],tile2_train[:-6],tile3_train[:-6],
                                                                  tile4_train[:-6],
                                  remote_weather_train[:-6],local_pm,
                            sample_batch,chan,chan2,chan3,time_t,time_step),
                                 steps = sample_per_epoch, verbose=1,workers=0)



print(predictions_edit.shape)
np.save('middlelayer_train_3day_4tile.npy',predictions_edit)
#np.save('prediction14_4hr_ver2.npy',pred_graph)